In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Get all files inside job_data folder
from os import listdir
from os.path import isfile, join
import os

cwd = os.getcwd()
mypath= join(cwd,"job_data")

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
dfs = []
for files in onlyfiles:
    temp_df = pd.read_csv("job_data/" + files)
    dfs.append(temp_df)

df = pd.concat(dfs,ignore_index=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4055 entries, 0 to 4054
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         4055 non-null   int64  
 1   keyword                            4055 non-null   object 
 2   link                               4055 non-null   object 
 3   job_id                             4055 non-null   int64  
 4   job_title                          4055 non-null   object 
 5   job_expired                        4055 non-null   bool   
 6   job_confidential                   4055 non-null   bool   
 7   job_salary_min                     842 non-null    float64
 8   job_salary_max                     842 non-null    float64
 9   job_salary_currency                3864 non-null   object 
 10  company                            4035 non-null   object 
 11  job_post_date                      4055 non-null   objec

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4055 entries, 0 to 4054
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         4055 non-null   int64  
 1   keyword                            4055 non-null   object 
 2   link                               4055 non-null   object 
 3   job_id                             4055 non-null   int64  
 4   job_title                          4055 non-null   object 
 5   job_expired                        4055 non-null   bool   
 6   job_confidential                   4055 non-null   bool   
 7   job_salary_min                     842 non-null    float64
 8   job_salary_max                     842 non-null    float64
 9   job_salary_currency                3864 non-null   object 
 10  company                            4035 non-null   object 
 11  job_post_date                      4055 non-null   objec

### Clean Job Title

In [6]:
import re

def clean_job_title(job_title):
    job_title = re.sub("[\(\[].*?[\)\]]", "", job_title) #remove everything within the bracket
    job_title = re.sub(r'[^\w]', ' ', job_title) #remove that's not alphanumeric or underscore
    job_title = re.sub(r'\w*\d\w*', '', job_title) #remove words with number
    job_title = job_title.strip() #remove trailing and ending whitespace
    job_title = re.sub("\s\s+", " ", job_title) #remove multiple whitespace
    job_title = job_title.title() #proper casing
    return job_title

In [7]:
df["clean_job_title"] = df["job_title"].apply(clean_job_title)

### Clean Job Description

In [8]:
# from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\teo-e\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\teo-e\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
#add stopwords
ALL_STOPWORDS = stopwords.words('english')
ALL_STOPWORDS += ["related", "good", "public", "ntu", "invite", "fellow", "new", "additional", "requirements", "knowledge", "work","experience", "skills", "job", "key", "responsibilities", "company", "shortlisted", "contract", "strong", "candidate", "regret", "contacted", "notified"]

In [10]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def clean_job_description(job_description):
#     soup = BeautifulSoup(job_description) #certain text merged together due to HTML structure
#     job_description = soup.get_text() #remove all html tags
    job_description = job_description.lower()
    job_description = re.sub('<[^<]+?>', ' ', job_description) #remove all html tags
    job_description = re.sub(r'[^\w]', ' ', job_description) #remove that's not alphanumeric or underscore
    job_description = re.sub(r'\d', ' ', job_description) #remove number
    job_description = job_description.strip() #remove trailing and ending whitespace
    job_description = re.sub(r"\s\s+", " ", job_description) #remove multiple whitespace
    text_tokens = job_description.split(" ")
    job_description = [lemmatizer.lemmatize(word) for word in text_tokens if word not in ALL_STOPWORDS] #stop word removal
    return " ".join(job_description)

In [11]:
def splitDescriptionAndRequirement(df):
    jd = df["job_description"]
    sjd = jd.split(">Requirements")
    if len(sjd) == 1:
        sjd = jd.split(">Job Requirements")
    df['clean_job_desc'] = clean_job_description(jd)
    df['job_desc_only'] = clean_job_description(sjd[0])
    df['job_req_only'] = clean_job_description("".join(sjd[1:]))
    return df

In [12]:
df = df.apply(splitDescriptionAndRequirement, axis=1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4055 entries, 0 to 4054
Data columns (total 40 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         4055 non-null   int64  
 1   keyword                            4055 non-null   object 
 2   link                               4055 non-null   object 
 3   job_id                             4055 non-null   int64  
 4   job_title                          4055 non-null   object 
 5   job_expired                        4055 non-null   bool   
 6   job_confidential                   4055 non-null   bool   
 7   job_salary_min                     842 non-null    float64
 8   job_salary_max                     842 non-null    float64
 9   job_salary_currency                3864 non-null   object 
 10  company                            4035 non-null   object 
 11  job_post_date                      4055 non-null   objec

In [14]:
df.to_csv("31_mar_clean_all_job_data.csv")